In [ ]:
import numpy as np
from gensim.models import Word2Vec
import string
import json
import nltk
import re
import pickle
import pandas as pd 
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout,SpatialDropout1D, Bidirectional
from keras.layers import Conv1D,MaxPooling1D,Dropout,Activation,Flatten,GlobalMaxPool1D,Reshape
from keras.models import Model
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
import numpy as np
import keras.backend.tensorflow_backend as backend
from keras.callbacks import TensorBoard
import tensorflow as tf
from collections import deque
import time
import random
from tqdm import tqdm
import os
from PIL import Image
import cv2


In [ ]:
DISCOUNT = 0.99
REPLAY_MEMORY_SIZE = 50  # How many last steps to keep for model training
MIN_REPLAY_MEMORY_SIZE = 35  # Minimum number of steps in a memory to start training
MINIBATCH_SIZE = 30  # How many steps (samples) to use for training
UPDATE_TARGET_EVERY = 10  # Terminal states (end of episodes)
MODEL_NAME = 'rekt'
MIN_REWARD = -7  # For model save
MEMORY_FRACTION = 0.20

# Environment settings
EPISODES = 250

epsilon = 1 # not a constant, going to be decayed
EPSILON_DECAY = 0.99998
MIN_EPSILON = 0.001

AGGREGATE_STATS_EVERY = 5  # episodes
SHOW_PREVIEW = False

In [ ]:
# For stats
ep_rewards = [-7]

# For more repetitive results
random.seed(1)
np.random.seed(1)
tf.random.set_seed(1)

# Memory fraction, used mostly when trai8ning multiple agents
#gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=MEMORY_FRACTION)
#backend.set_session(tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)))

# Create models folder
if not os.path.isdir('models'):
    os.makedirs('models')


# Own Tensorboard class
class ModifiedTensorBoard(TensorBoard):

    # Overriding init to set initial step and writer (we want one log file for all .fit() calls)
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.step = 1
        self.writer =tf.summary.create_file_writer(self.log_dir)

    # Overriding this method to stop creating default log writer
    def set_model(self, model):
        pass

    # Overrided, saves logs with our step number
    # (otherwise every .fit() will start writing from 0th step)
    def on_epoch_end(self, epoch, logs=None):
        self.update_stats(**logs)

    # Overrided
    # We train for one batch only, no need to save anything at epoch end
    def on_batch_end(self, batch, logs=None):
        pass

    # Overrided, so won't close writer
    def on_train_end(self, _):
        pass

    # Custom method for saving own metrics
    # Creates writer, writes custom metrics and closes writer
    def update_stats(self, **stats):
        self._write_logs(stats, self.step)



In [ ]:
class QueryEnv:
    ACTION_SPACE_SIZE = 0
    num_of_query = 0
    def __init__(self):
        schema = "C:/Kshitij/DEV/Datasets/spider/database/college_1/TinyCollege.sql"
        fd = open(schema, 'r',encoding='utf-8')
        sqlFile = fd.read()
        regex = re.compile('CREATE TABLE [^(]*', re.IGNORECASE)
        tabs = re.findall(regex,sqlFile)
        tabs1 = [i[13:] for i in tabs]
        table_names = [re.sub('[^A-Za-z0-9]+', '', i) for i in tabs1]
        self.action_space = table_names
        self.label = {}
        QueryEnv.ACTION_SPACE_SIZE = len(self.action_space)
        for i in range(len(self.action_space)):
            base = [0]*len(self.action_space)
            base[i] = 1
            self.label[self.action_space[i]] = base
    
    def reset(self):
        filename = "C:\\Kshitij\\DEV\\Datasets\\spider\\train_spider.json"
        # data_Set = open("C:\\Kshitij\\DEV\\Datasets\\spider\\train_others.json")
        with open(filename, 'r') as f:
            datastore = json.load(f)
        queries = []
        questions = []
        table = "college_1"
        for i in range(len(datastore)):
            if(datastore[i]['db_id'] == table):
                queries.append(datastore[i]['query'])
                questions.append(datastore[i]['question'])
        
        self.OGquery = queries[QueryEnv.num_of_query]
        query_state = self.MKL_convert(queries[QueryEnv.num_of_query])
        self.state_query = query_state
        
        self.episode_step = 0
        return query_state
        
    def MKL_convert(self,x):
        num = 0
        dbs = x
        SQL_key = ['FROM', 'REPLACE', 'SELECT', 'INSERT', 'AS', 'IS', 'TRUNCATE', 'EXISTS', 'VALUES', 'PRIMARY', 'MAX', 'UNION', 'A', 'TABLE', 'JOIN', 'UPDATE', 'ALL', 'OR', 'NULL', 'ALTER', 'TOP', 'AND', 'DELETE', 'LEFT', 'IN', 'ANY', 'SQL', 'RIGHT', 'HAVING', 'KEY', 'DATABASE', 'DESC', 'EXEC', 'COUNT', 'CONSTRAINT', 'INDEX', 'GROUP', 'PROCEDURE', 'FOREIGN', 'LIMIT', 'BACKUP', 'OUTER', 'ORDER', 'WHERE', 'DEFAULT', 'DROP', 'CHECK', 'NOT', 'ROWNUM', 'ADD', 'AVG', 'MIN', 'INTO', 'CASE', 'FULL', 'VIEW', 'DISTINCT', 'ASC', 'UNIQUE', 'SUM', 'LIKE', 'BY', 'BETWEEN', 'INNER', 'COLUMN', 'SET', 'CREATE']
        for i in dbs.split(" "):
            if(i.upper() in self.action_space):
                dbs = dbs.replace(i,"[mark]")
                num += 1
        query = dbs
        self.MAX_REWARD = num
        return query        
    
    def step(self,action_index):
        self.episode_step += 1
        reward = 0
        action = self.action_space[action_index]
        new_query = ''
        change_word_loc = 0
        # print(state_query)
        flag = 0
        for i in self.state_query.split(" "):
            if(flag == 0):
                if(i == "[mark]"):
                    flag = 1
                    change_word_loc = i
                    new_query += action+" "
                else:
                    new_query += i+" "
            else:
                new_query += i+" "
        self.state_query = new_query
#         print(new_query)
        if(self.OGquery == new_query):
            reward = self.MAX_REWARD
        else:
            query_std = self.OGquery.split(" ")
            for i in range(len(query_std)):
                if(i == change_word_loc):
                    if(query_std[i] == new_query.split(" ")[i]):
                        reward = 1
                    else:
                        reward = -1
        done = False
        if(reward == self.MAX_REWARD or self.episode_step >= 200):
            done = True
        
        return new_query,reward,done
    
    def render(self):
        print("THIS IS THE STATE OF QUERY:-",self.state_query)

env = QueryEnv()

In [ ]:
class DQNAgent:
    def __init__(self):
        
        pickle_in = open("word_vectors_college_1.pickle","rb")
        self.word_vectors = pickle.load(pickle_in)
        pickle_in = open("word_index_college_1.pickle","rb")
        self.word_index = pickle.load(pickle_in)
        
        self.MAX_NB_WORDS = len(self.word_vectors.vocab)
        self.MAX_SEQUENCE_LENGTH = 55
        self.nb_words = min(self.MAX_NB_WORDS, len(self.word_vectors.vocab))
        self.WV_DIM = 100
        # we initialize the matrix with random numbers
        self.wv_matrix = (np.random.rand(self.nb_words+1, self.WV_DIM) - 0.5) / 5.0
        for word, i in self.word_index.items():
            if i >= self.MAX_NB_WORDS:
                continue
            try:
                embedding_vector = self.word_vectors[word]
                # words not found in embedding index will be all-zeros.
                self.wv_matrix[i] = embedding_vector
            except:
                pass

        # Main model
        self.model = self.create_model()

        # Target network
        self.target_model = self.create_model()
        self.target_model.set_weights(self.model.get_weights())

        # An array with last n steps for training
        self.replay_memory = deque(maxlen=REPLAY_MEMORY_SIZE)

        # Custom tensorboard object
        self.tensorboard = ModifiedTensorBoard(log_dir="logs/{}-{}".format(MODEL_NAME, int(time.time())))

        # Used to count when to update target network with main network's weights
        self.target_update_counter = 0
    
    def preprocess(self,x):
        pred_tab = x
        
        sequences_pred = [[self.word_index.get(t, 0) for t in query] for query in pred_tab]
        data_pred = pad_sequences(sequences_pred, maxlen=self.MAX_SEQUENCE_LENGTH,padding="pre")

        return data_pred
    
    def create_model(self):
        wv_layer = Embedding(self.nb_words + 1, self.WV_DIM, weights=[self.wv_matrix], input_length=55,trainable=False)
        query_input = Input(shape=(self.MAX_SEQUENCE_LENGTH,), dtype='int32') 
        embedded_sequences = wv_layer(query_input)
        x = Bidirectional(LSTM(175, return_sequences=True, dropout=0.25, recurrent_dropout=0.1))(embedded_sequences)
        x = GlobalMaxPool1D()(x)
        x = Dense(100 , activation="relu")(x)
        preds = Dense(QueryEnv.ACTION_SPACE_SIZE, activation="softmax")(x)
        model = Model(inputs=[query_input], outputs=preds)
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model
    
    # Adds step's data to a memory replay array
    # (observation space, action, reward, new observation space, done)
    def update_replay_memory(self, transition):
        self.replay_memory.append(transition)
    # Trains main network every step during episode
    def train(self, terminal_state, step):
        # Start training only if certain number of samples is already saved
        if len(self.replay_memory) < MIN_REPLAY_MEMORY_SIZE:
            return
        # Get a minibatch of random samples from memory replay table
        minibatch = random.sample(self.replay_memory, MINIBATCH_SIZE)
        
       # Get current states from minibatch, then query NN model for Q values
#         print(minibatch)
        current_states = np.array(self.preprocess([transition[0] for transition in minibatch]))
#         print(current_states.shape)
        print("PREDICTING THE CURRENT Q VALUE")
        current_qs_list = self.model.predict(current_states)

        # Get future states from minibatch, then query NN model for Q values
        # When using target network, query it, otherwise main network should be queried
        new_current_states = np.array(self.preprocess([transition[3] for transition in minibatch]))
        print("PREDICTING THE FUTURE Q VALUE")
        future_qs_list = self.target_model.predict(new_current_states)

        X = []
        y = []
        # Now we need to enumerate our batches
        for index, (current_state, action, reward, new_current_state, done) in enumerate(minibatch):

            # If not a terminal state, get new q from future states, otherwise set it to 0
            # almost like with Q Learning, but we use just part of equation here
            if not done:
                max_future_q = np.max(future_qs_list[index])
                new_q = reward + DISCOUNT * max_future_q
            else:
                new_q = reward

            # Update Q value for given state
            current_qs = current_qs_list[index]
            current_qs[action] = new_q

            # And append to our training data
            X.append(current_state)
            y.append(current_qs)

        # Fit on all samples as one batch, log only on terminal state
        print("FITING THE NEW Q VALUE")
        self.model.fit(np.array(self.preprocess(X)), np.array(y), batch_size=MINIBATCH_SIZE, verbose=0, shuffle=False)

        # Update target network counter every episode
        if terminal_state:
            self.target_update_counter += 1

        # If counter reaches set value, update target network with weights of main network
        if self.target_update_counter > UPDATE_TARGET_EVERY:
            self.target_model.set_weights(self.model.get_weights())
            self.target_update_counter = 0

    # Queries main network for Q values given current observation space (environment state)
    def get_qs(self, state):
        print("PREDICTING THE Q VALUE")
        pred_q = self.model.predict(self.preprocess(state))
#         print(pred_q.shape)
        return pred_q

agent = DQNAgent()

In [ ]:
# Iterate over episodes
for episode in tqdm(range(1, EPISODES + 1), ascii=True, unit='episodes'):

    # Update tensorboard step every episode
    agent.tensorboard.step = episode

    # Restarting episode - reset episode reward and step number
    episode_reward = 0
    step = 1
    
    # Reset environment and get initial state
    current_state = env.reset()

    # Reset flag and start iterating until episode ends
    done = False
    while not done:

        # This part stays mostly the same, the change is to query a model for Q values
        if np.random.random() > epsilon:
            # Get action from Q table
            action = np.argmax(agent.get_qs(current_state))
            print("Exploitation:",action)
        else:
            # Get random action
            action = np.random.randint(0, QueryEnv.ACTION_SPACE_SIZE)
            print("Exploration:",action)

        new_state, reward, done = env.step(action)

        # Transform new continous state to new discrete state and count reward
        episode_reward += reward

        if SHOW_PREVIEW and not episode % AGGREGATE_STATS_EVERY:
            env.render()

        # Every step we update replay memory and train main network
        print(current_state, action, reward, new_state, done)
        agent.update_replay_memory((current_state, action, reward, new_state, done))
        agent.train(done, step)

        current_state = new_state
        step += 1

    # Append episode reward to a list and log stats (every given number of episodes)
    ep_rewards.append(episode_reward)
    if not episode % AGGREGATE_STATS_EVERY or episode == 1:
        average_reward = sum(ep_rewards[-AGGREGATE_STATS_EVERY:])/len(ep_rewards[-AGGREGATE_STATS_EVERY:])
        min_reward = min(ep_rewards[-AGGREGATE_STATS_EVERY:])
        max_reward = max(ep_rewards[-AGGREGATE_STATS_EVERY:])
#         agent.tensorboard.update_stats(reward_avg=average_reward, reward_min=min_reward, reward_max=max_reward, epsilon=epsilon)

        # Save model, but only when min reward is greater or equal a set value
        if min_reward >= MIN_REWARD:
            agent.model.save(f'models/{MODEL_NAME}__{max_reward:_>7.2f}max_{average_reward:_>7.2f}avg_{min_reward:_>7.2f}min__{int(time.time())}.model')

    # Decay epsilon
    if epsilon > MIN_EPSILON:
        epsilon *= EPSILON_DECAY
        epsilon = max(MIN_EPSILON, epsilon)

In [ ]:
action = np.argmax(agent.get_qs(current_state))
print(action)

In [ ]:
# minibatch = random.sample(agent.replay_memory, MINIBATCH_SIZE)
# for i in minibatch:
#         print(i)
print(agent.replay_memory)